<a href="https://colab.research.google.com/github/RaghavSrivastava25/Sentiment-Analysis-of-Tweets-Predicting-and-Visualizing-Public-Opinion/blob/main/Tweets_Sentiment_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import tensorflow as tf
import multiprocessing
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv("/content/drive/MyDrive/ML/All_Tweets.csv")

<ipython-input-5-0be622c7f935>:1: DtypeWarning: Columns (0,5,7,8,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/ML/All_Tweets.csv")


In [ ]:
df.shape

(252350, 17)

In [ ]:
df.describe()

,Unnamed: 0,sentiment_analysis,tweets_ID,tweets_month,tweet_link,Top_Events,tweet_for,events_flag,tweet_ner_flag,tweet_text,tweet_by,event_analysis,crawled_date,events_detection_flag,All_Events,ner_analysis,tweet_date
count,252350,13729,252337,252341,252341,162843,252338,252341,252337,252337,252333,150796,252333,211744,211728,7264,252333
unique,252350,10263,252337,23,227949,20175,86,8,2,222697,38412,21280,110883,4,31399,3737,224376
top,0,"[None, None]",Axis Bank Ltd +UNIQUE_T_AND_N_ID_SEPARATOR+ ht...,2022-12,https://twitter.com/Zerotoone20/status/1636113...,Null,Mahindra & Mahindra Limited,False,True,"@lClClBank Hi, we are here to assist customers...",ICICIBank_Care,[],2022-07-20 17:36:49,1,Null,[],2023-03-21 08:39:17
freq,1,239,1,31223,13,107160,45235,111861,252336,116,9056,124036,20365,179884,90890,1149,18


In [ ]:
df.head(5)

,Unnamed: 0,sentiment_analysis,tweets_ID,tweets_month,tweet_link,Top_Events,tweet_for,events_flag,tweet_ner_flag,tweet_text,tweet_by,event_analysis,crawled_date,events_detection_flag,All_Events,ner_analysis,tweet_date
0,0,NaN,Axis Bank Ltd +UNIQUE_T_AND_N_ID_SEPARATOR+ ht...,2022-06,https://twitter.com/21Himanshugupta/status/153...,NaN,Axis Bank Ltd,False,True,@PankajChandak_ @ZeeBusiness Huge short coveri...,21Himanshugupta,[],2022-06-21 14:24:26,1.0,[],NaN,2022-06-21 12:55:29
1,1,NaN,Axis Bank Ltd +UNIQUE_T_AND_N_ID_SEPARATOR+ ht...,2022-06,https://twitter.com/AlertTrade/status/15351422...,NaN,Axis Bank Ltd,False,True,Trading Ideas: Bullish in Axis bank https://t....,AlertTrade,[],2022-06-10 14:24:25,1.0,[],NaN,2022-06-10 11:40:04
2,2,NaN,Axis Bank Ltd +UNIQUE_T_AND_N_ID_SEPARATOR+ ht...,2022-06,https://twitter.com/AlertTrade/status/15374897...,NaN,Axis Bank Ltd,False,True,Trading Ideas: Axis bank formed a head and sho...,AlertTrade,[],2022-06-16 23:24:40,1.0,[],NaN,2022-06-16 23:08:12
3,3,NaN,Axis Bank Ltd +UNIQUE_T_AND_N_ID_SEPARATOR+ ht...,2022-06,https://twitter.com/AlertTrade/status/15411712...,NaN,Axis Bank Ltd,False,True,Trading Ideas: AXIS BANK (SELL) https://t.co/S...,AlertTrade,[],2022-06-27 05:24:23,1.0,[],NaN,2022-06-27 02:57:08
4,4,NaN,Axis Bank Ltd +UNIQUE_T_AND_N_ID_SEPARATOR+ ht...,2022-06,https://twitter.com/AndreasSteno/status/153234...,NaN,Axis Bank Ltd,False,True,"@GaryHaubold We are just going from a ""negativ...",AndreasSteno,[],2022-06-02 20:24:59,1.0,[],NaN,2022-06-02 18:23:18


In [ ]:
df = df[['tweet_for', 'tweet_date', 'tweet_text']]
df

,tweet_for,tweet_date,tweet_text
0,Axis Bank Ltd,2022-06-21 12:55:29,@PankajChandak_ @ZeeBusiness Huge short coveri...
1,Axis Bank Ltd,2022-06-10 11:40:04,Trading Ideas: Bullish in Axis bank https://t....
2,Axis Bank Ltd,2022-06-16 23:08:12,Trading Ideas: Axis bank formed a head and sho...
3,Axis Bank Ltd,2022-06-27 02:57:08,Trading Ideas: AXIS BANK (SELL) https://t.co/S...
4,Axis Bank Ltd,2022-06-02 18:23:18,"@GaryHaubold We are just going from a ""negativ..."
...,...,...,...
252345,Wipro Ltd,2022-09-13 18:23:29,Goldman double upgrades Wipro to Buy on attrac...
252346,Wipro Ltd,2022-09-17 15:00:40,Current SIP stocks in IT sector. Invest and fo...
252347,Wipro Ltd,2022-09-22 14:40:06,Wipro Foundation's Santoor Scholarship Program...
252348,Wipro Ltd,2022-09-04 16:28:04,Python Developer at Wipro\nAbout Wipro Limited...


In [ ]:
companies = []
for company in df['tweet_for'].unique():
  companies.append(company)

In [ ]:
del companies[83]

In [ ]:
companies.append('IndusInd Bank')
companies.append('Japanese Construction Equipment Cos')

In [ ]:
df

,tweet_for,tweet_date,tweet_text
0,Axis Bank Ltd,2022-06-21 12:55:29,@PankajChandak_ @ZeeBusiness Huge short coveri...
1,Axis Bank Ltd,2022-06-10 11:40:04,Trading Ideas: Bullish in Axis bank https://t....
2,Axis Bank Ltd,2022-06-16 23:08:12,Trading Ideas: Axis bank formed a head and sho...
3,Axis Bank Ltd,2022-06-27 02:57:08,Trading Ideas: AXIS BANK (SELL) https://t.co/S...
4,Axis Bank Ltd,2022-06-02 18:23:18,"@GaryHaubold We are just going from a ""negativ..."
...,...,...,...
252345,Wipro Ltd,2022-09-13 18:23:29,Goldman double upgrades Wipro to Buy on attrac...
252346,Wipro Ltd,2022-09-17 15:00:40,Current SIP stocks in IT sector. Invest and fo...
252347,Wipro Ltd,2022-09-22 14:40:06,Wipro Foundation's Santoor Scholarship Program...
252348,Wipro Ltd,2022-09-04 16:28:04,Python Developer at Wipro\nAbout Wipro Limited...


In [ ]:
df['tweet_date'] = pd.to_datetime(df['tweet_date'])
df = df.sort_values('tweet_date')

print("Earliest Date:", df['tweet_date'].min())
print("Latest Date:", df['tweet_date'].max())

Earliest Date: 2021-10-19 01:45:00
Latest Date: 2023-04-21 02:21:44


In [ ]:
df

,tweet_for,tweet_date,tweet_text
19255,HDFC Bank Ltd,2021-10-19 01:45:00,#HDFCBank’s bullishness on balance sheet growt...
19256,HDFC Bank Ltd,2021-10-19 06:43:57,HDFC Bank | Brokerage: Morgan Stanley | Rating...
19257,HDFC Bank Ltd,2021-10-19 07:55:59,[Automated: 19-Oct 07:55 AM]\nIndian ADRs on U...
19258,HDFC Bank Ltd,2021-10-19 08:27:29,Good Morning Everyone. Few queries abt the cal...
19259,HDFC Bank Ltd,2021-10-19 08:29:00,"Good Morning Team, 18.5 K &amp; 40 K is done....."
...,...,...,...
233439,Oil & Natural Gas Corporation Ltd,NaT,🛢 Today's Crude Oil Prices:
233440,NaN,NaT,NaN
233441,NaN,NaT,NaN
233442,NaN,NaT,NaN


In [ ]:
max_len = len(df)
print(max_len)

252350


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_gpus = torch.cuda.device_count()

if num_gpus > 1:
    model = nn.DataParallel(model)

batch_size = num_gpus * 2

def perform_sentiment_analysis(text):
    tokens = tokenizer.batch_encode_plus(
        text,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids = tokens['input_ids'].to(device)
    attention_mask = tokens['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

    return predictions.cpu().tolist()

In [ ]:
df['sentiments'] = [None] * len(df)

for i in range(0, len(df), batch_size):
    batch_tweets = df['tweet_text'][i:i+batch_size].tolist()
    batch_tweets = [str(tweet) for tweet in batch_tweets]

    batch_sentiments = perform_sentiment_analysis(batch_tweets)
    df['sentiments'][i:i+batch_size] = batch_sentiments

sentiment_mapping = {2: 0, 0: -1, 1: 1}
df['sentiments'] = df['sentiments'].map(sentiment_mapping)

<ipython-input-24-f0c74511aaa7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiments'][i:i+batch_size] = batch_sentiments


In [ ]:
sentiment_counts = df['sentiments'].value_counts()

In [ ]:
df

,tweet_for,tweet_date,tweet_text,sentiments
19255,HDFC Bank Ltd,2021-10-19 01:45:00,#HDFCBank’s bullishness on balance sheet growt...,-1
19256,HDFC Bank Ltd,2021-10-19 06:43:57,HDFC Bank | Brokerage: Morgan Stanley | Rating...,-1
19257,HDFC Bank Ltd,2021-10-19 07:55:59,[Automated: 19-Oct 07:55 AM]\nIndian ADRs on U...,0
19258,HDFC Bank Ltd,2021-10-19 08:27:29,Good Morning Everyone. Few queries abt the cal...,0
19259,HDFC Bank Ltd,2021-10-19 08:29:00,"Good Morning Team, 18.5 K &amp; 40 K is done.....",1
...,...,...,...,...
233439,Oil & Natural Gas Corporation Ltd,NaT,🛢 Today's Crude Oil Prices:,0
233440,NaN,NaT,NaN,0
233441,NaN,NaT,NaN,0
233442,NaN,NaT,NaN,0


In [ ]:
output_file = '/content/drive/MyDrive/final_dataframe.csv'
df.to_csv(output_file, index=False)

In [ ]:
fdf = pd.read_csv("/content/drive/MyDrive/final_dataframe.csv")

In [ ]:
fdf

,tweet_for,tweet_date,tweet_text,sentiments
0,HDFC Bank Ltd,2021-10-19 01:45:00,#HDFCBank’s bullishness on balance sheet growt...,-1
1,HDFC Bank Ltd,2021-10-19 06:43:57,HDFC Bank | Brokerage: Morgan Stanley | Rating...,-1
2,HDFC Bank Ltd,2021-10-19 07:55:59,[Automated: 19-Oct 07:55 AM]\nIndian ADRs on U...,0
3,HDFC Bank Ltd,2021-10-19 08:27:29,Good Morning Everyone. Few queries abt the cal...,0
4,HDFC Bank Ltd,2021-10-19 08:29:00,"Good Morning Team, 18.5 K &amp; 40 K is done.....",1
...,...,...,...,...
252345,Oil & Natural Gas Corporation Ltd,NaN,🛢 Today's Crude Oil Prices:,0
252346,NaN,NaN,NaN,0
252347,NaN,NaN,NaN,0
252348,NaN,NaN,NaN,0


In [ ]:
fdf['tweet_date'] = pd.to_datetime(fdf['tweet_date'])

fdf['tweet_date'] = fdf['tweet_date'].dt.date

average_sentiments = fdf.groupby('tweet_date')['sentiments'].mean().reset_index(name='average_sentiment')

new_df = average_sentiments.copy()

new_df['average_sentiment'] = new_df['average_sentiment'].apply(lambda x: -1 if x < 0 else 1 if x > 0 else 0)

new_df

,tweet_date,average_sentiment
0,2021-10-19,1
1,2021-10-21,-1
2,2021-10-22,-1
3,2021-10-23,1
4,2021-10-24,-1
...,...,...
539,2023-04-17,1
540,2023-04-18,1
541,2023-04-19,1
542,2023-04-20,-1


In [ ]:
output_file = '/content/drive/MyDrive/output_dataframe.csv'
new_df.to_csv(output_file, index=False)